<a href="https://colab.research.google.com/github/Zeepratik4444/Snippents_ML/blob/main/Deep_Learning_HelpBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Genral class module for torchVision

In [ ]:
from collections.abc import Sequence
from torch import nn
class FashionMNISTV2(nn.Module):
  """
  Model arcitechture that replicates the TinyVGG
  Model from CNN explainer website.
  """
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.conv_block_1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv_block_2=nn.Sequential(
      nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
      nn.ReLU(),
      nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,kernel_size=3,stride=1,padding=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2)
    )
    self.classifier=nn.Sequential(
      nn.Flatten(),
      nn.Linear(in_features=hidden_units*7*7,out_features=output_shape)
    )

  def forward(self,x):
    x=self.conv_block_1(x)
    x=self.conv_block_2(x)
    x=  self.classifier(x)
    return(x)



# 2. Training function for training model and let it learn.

In [ ]:
def train_mode(model:torch.nn.Module,
               train_data_loader:torch.utils.data.DataLoader,
               test_data_loader: torch.utils.data.DataLoader,
               loss: torch.nn.Module,
               accuracy_fn,
               epochs: int,
               device: torch.device = device,
               ):
  start_time=timer()
  total_loss,total_acc=0,0
  model.to(device)
  for epoch in tqdm(range(epochs)):
    for batch ,(x_train,y_train) in enumerate(train_data_loader):
      x_train,y_train=x_train.to(device),y_train.to(device)
      model.train()
      y_pred=model(x_train)
      loss= loss_fn(y_pred,y_train)
      total_loss+=loss
      total_acc+=accuracy_fn(y_train,y_pred.argmax(dim=1))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    total_loss/=len(train_data_loader)
    total_acc/=len(train_data_loader)
    if batch % 400==0:
      print(f"Looked at {batch* len(x)}/{len(train_data.dataset)} samples.")
    #Testing
    test_loss,test_acc=0,0
    with torch.inference_mode():
      for x_test, y_test in test_data_loader:
        # Send data to GPU
        x_test, y_test = x_test.to(device), y_test.to(device)

        # 1. Forward pass
        test_pred = model(x_test)

        # 2. Calculate loss and accuracy
        test_loss += loss_fn(test_pred, y_test)
        test_acc += accuracy_fn(y_true=y_test,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
      test_loss/= len(test_data_loader)
      test_acc/=len(test_data_loader)
  end_time=timer()
  time_taken=end_time-start_time

  print(f"Train Loss : {total_loss:.5f} , Train Acc : {total_acc} \n Test Loss :{test_loss:.5f}  ,Test Acc : {test_acc:.2f}% \n Time Taken : {time_taken:.3f} Seconds")




# 3. To make torchvision predictions

In [ ]:
def make_predictions(model:torch.nn.Module,
                     data: list,
                     device: torch.device=device):
  pred_probs=[]
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data:
      # Preparing the sample( add a batch dimension and pass to target device)
      sample=torch.unsqueeze(sample,dim=0).to(device)
      # FOrward pass (model outputs raw logits)
      pred_logits=model(sample)
      # get prediction probability ( logit-> prediction probability)
      pred_prob= torch.softmax(pred_logits.squeeze(),dim=0)
      # Get pred_prob off the GPU for further calcualtions
      pred_probs.append(pred_prob.cpu())
 # Stack the pred_probs to turn list into a tesnor
  return torch.stack(pred_probs)